In [1]:
import keras
from keras import backend as K
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.applications.vgg16 import VGG16
import os
import cv2
import glob
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from keras.layers import Dense
from tensorflow.keras.layers import Conv2D, Flatten, MaxPool2D, Dropout, BatchNormalization ,Activation , MaxPooling2D
from keras.models import Sequential
from keras import layers as ls
from keras.models import Model 
from keras.applications.inception_v3 import InceptionV3
from keras.layers import concatenate
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
import numpy as np 
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model

In [2]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

x = base_model.output
x = ls.Flatten()(x)
x = Dense(28, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)


for layer in model.layers[0:20]:
    layer.trainable = False
    
    
early = EarlyStopping(monitor = 'val_accuracy', min_delta = 0, 
                      patience = 10, verbose= 1 , mode = 'auto')


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


model.load_weights("arsl-2.h5")

In [3]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [4]:

import time
import numpy as np
from keras.models import model_from_json
import operator
import cv2
import sys, os
from keras.models import load_model
import mediapipe as mp
import uuid


import arabic_reshaper
from bidi.algorithm import get_display

from PIL import ImageFont, ImageDraw, Image
#================================================
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
#================================================
output_Phrase=""


count_of_writen_letter=0
l=[]
numframe=0
count=0
arabic_imge_width=800
arabic_imge_hight=400
how_many_count_to_write_the_letter=2
how_many_count_frame_to_predict=20
cap = cv2.VideoCapture(0)
cap.set(3,3000)
cap.set(4,3000)
w=cap.get(3)
h=cap.get(4)

# print(w)
# print(h)

sign_image=np.ones((1500,1500))
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands: 
    while cap.isOpened():

        start=time.time()
        ret, frame = cap.read()
        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Flip on horizontal
        image = cv2.flip(image, 1)
        # Set flag
        image.flags.writeable = False
        # Detections
        results = hands.process(image)
        # Set flag to true
        image.flags.writeable = True
        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        # Rendering results
        if results.multi_hand_landmarks:
            for hand in results.multi_hand_landmarks:
                xl=[]
                yl=[]
                zl=[]
                for point in hand.landmark:
                    xl.append(point.x)
                    yl.append(point.y)
                    zl.append(point.z)
                x1=int(min(xl)*w)-80
                y1=int(min(yl)*h)-50
                x2=int(max(xl)*w)+50
                y2=int(max(yl)*h)+50
                if x1<0:
                    x1=0
                if y1<0:
                    y1=0

            cv2.rectangle(image,(x1,y1),(x2,y2),(0,0,255),1)
            #sign_image
            sign_image=image[y1:y2,x1:x2]
            try:
                   
                if numframe%how_many_count_frame_to_predict==0:
    
                    sign_image = cv2.resize(sign_image, (128, 128))               
                    sign_image=sign_image.reshape(1, 128, 128, 3)
                    result = model.predict(sign_image)

                    #English mapping
                    #=========================================================================
                    prediction1 = { '7a2'  : result[0][0],
                                    'ain'  : result[0][1],
                                    'alef' : result[0][2],
                                    'ba2'  : result[0][3],
                                    'daal' : result[0][4],
                                    'dad'  : result[0][5],
                                    'faa'  : result[0][6],
                                    'gem'  : result[0][7],
                                    'ghain': result[0][8],
                                    'haa'  : result[0][9],
                                    'kaaf' : result[0][10],
                                    'kha'  : result[0][11],
                                    'laam' : result[0][12],
                                    'meem' : result[0][13],
                                    'noon' : result[0][14],
                                    'qaaf' : result[0][15],
                                    'ra2'  : result[0][16],
                                    'sad'  : result[0][17],
                                    'seen' : result[0][18],
                                    'sheen': result[0][19],
                                    'ta2'  : result[0][20],
                                    'tah'  : result[0][21],
                                    'tha2' : result[0][22],
                                    'waaw' : result[0][23],
                                    'yaa'  : result[0][24],
                                    'zaaa' : result[0][25],
                                    'zain' : result[0][26],
                                    'zal'  : result[0][27]
                    }
                    prediction2 = { 'ح'  : result[0][0],
                                    'ع'  : result[0][1],
                                    'ا' : result[0][2],
                                    'ب'  : result[0][3],
                                    'د' : result[0][4],
                                    'ض'  : result[0][5],
                                    'ف'  : result[0][6],
                                    'ج'  : result[0][7],
                                    'غ': result[0][8],
                                    'ه'  : result[0][9],
                                    'ك' : result[0][10],
                                    'خ'  : result[0][11],
                                    'ل' : result[0][12],
                                    'م' : result[0][13],
                                    'ن' : result[0][14],
                                    'ق' : result[0][15],
                                    'ر'  : result[0][16],
                                    'ص'  : result[0][17],
                                    'س' : result[0][18],
                                    'ش': result[0][19],
                                    'ت'  : result[0][20],
                                    'ط'  : result[0][21],
                                    'ث' : result[0][22],
                                    'و' : result[0][23],
                                    'ي'  : result[0][24],
                                    'ظ' : result[0][25],
                                    'ز' : result[0][26],
                                    'ذ'  : result[0][27]
                    }
                    # Sorting based on top prediction
                    #=========================================================================
                    prediction1 = sorted(prediction1.items(), key=operator.itemgetter(1), reverse=True)
                    prediction2 = sorted(prediction2.items(), key=operator.itemgetter(1), reverse=True)
                    # Sorting based on top prediction
                    #append the list
                               

                    if(len(l)==0):
                        l.append(prediction1[0][0])
                        lcount=count
                    if prediction1[0][0]==l[-1]:
                        count=count+1
                    if count==how_many_count_to_write_the_letter:
                        output_Phrase=output_Phrase+prediction2[0][0]
                        count_of_writen_letter=count_of_writen_letter+1
                        count=0
                        l=[]
                    if count == lcount:
                        count=0
                    l.append(prediction1[0][0]) 

                cv2.putText(image, "{}".format(count_of_writen_letter),(550,300), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 2)
                cv2.putText(image, prediction1[0][0],(500,400), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 2) 
                cv2.imshow('Hand Tracking', image)
#                 print("frame")
#                 print(numframe)
                numframe=numframe+1
                #show the line of all letter
                #=========================================================================
                reshaped_text = arabic_reshaper.reshape(output_Phrase)
                bidi_text = get_display(reshaped_text) 
                fontpath = "arial.ttf"
                font = ImageFont.truetype(fontpath, 32)
            

                img=np.full([arabic_imge_hight,arabic_imge_width,3],0,np.uint8)
                cv2.rectangle(img,(0,0),(arabic_imge_width,arabic_imge_hight),(50,180,30),-1)
                img_pil = Image.fromarray(img)
                draw = ImageDraw.Draw(img_pil)
                draw.text((400,180),bidi_text, font = font)
                img = np.array(img_pil)              
                cv2.imshow('arabic image',img)
                

            except ValueError:
                print(ValueError)
                print("Error form predict area")
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
cap.release()
cv2.destroyAllWindows()
print(output_Phrase)

1/1 [==============================] - 0s 44ms/step


KeyboardInterrupt: 

: 